# Predicting the end of Coronavirus disease 2019

## Correlation between growth rate and types of mitigation across countries (china vs ROW)

### (When Covid-19 started 31 Dec 2019)

If we need to predict the end of covid-19 in China and the rest of the World, we must know how involve an active confirmed. We know that
> **active confirmed = confirmed - (death + recovered)**

1- The disease decrease if $\dfrac{d\small active confirmed}{dt} < 0 $ and $ \Delta recovered > \Delta death $ $\forall  t \in [0, \inf)$.

let's set ac = active confirmed  and introduce a growth rate for ac $\alpha(t)_{ac}$ in our equation above.

$\alpha_{ac}(t) = \dfrac{ac_{new-day} -  ac_{previous-day}}{ac_{previous-day}}$. So,

> $ \alpha_{ac}(t) < 0 $  and $ \dfrac{\Delta recovered}{\Delta death} > 1$,$\forall  t \in [0, \inf)$

The goal of this notebook is to find if the growth rate of active confirmed is become negative in china and somewhere in the rest of the world. And people who are active confirmed recover the health better and better.

# Update !!!!

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

To begin, I divide our data on two. one is based on China and another ROW. We start.

In [ ]:
#we import all package that we need
import matplotlib.pyplot as plt
import seaborn as sns 
import statsmodels as sm
import folium as fl
from pathlib import Path
from sklearn.impute import SimpleImputer
import plotly.offline as py
import plotly.express as px
import cufflinks as cf

In [ ]:
sns.set()
%matplotlib inline
pd.options.plotting.backend
py.init_notebook_mode(connected=False)
cf.set_config_file(offline=True)
sns.set()
pd.plotting.register_matplotlib_converters

**Clean data from covid_19_data.csv file**

In [ ]:
dataFile = '/kaggle/input/novel-corona-virus-2019-dataset/covid_19_data.csv'
covid = pd.read_csv(dataFile)

In [ ]:
#see data 
covid.head()

In [ ]:
# check if there exist a missing value
mis = covid.isnull().sum()
mis[mis>0]

Only Province/State have a missing value. I can impute it because this variable is necessary for visualizing a data.

In [ ]:
imputer = SimpleImputer(strategy='constant')#here I use constant because I cannot put another Province/State
#that we do not know or that does not correspond to his country/region  
impute_covid = pd.DataFrame(imputer.fit_transform(covid), columns=covid.columns)
impute_covid.head()

In [ ]:
#convert ObservationDate and Last Update object to datetime
#convert confirmed, recovered, death to numeric
impute_covid['ObservationDate'] = pd.to_datetime(impute_covid['ObservationDate'])
impute_covid['Last Update'] = pd.to_datetime(impute_covid['Last Update'])
impute_covid['Confirmed'] = pd.to_numeric(impute_covid['Confirmed'], errors='coerce')
impute_covid['Recovered'] = pd.to_numeric(impute_covid['Recovered'], errors='coerce')
impute_covid['Deaths'] = pd.to_numeric(impute_covid['Deaths'], errors='coerce')

In [ ]:
impute_covid.info()

# Some relevant Information and Visualization

we are going to visualize this data and make some statistics to find the relevant information.

In [ ]:
# see again data table
impute_covid.head(3)

In [ ]:
# we compute the active_confirmed
impute_covid['active_confirmed'] = impute_covid['Confirmed'].values - \
(impute_covid['Deaths'].values+impute_covid['Recovered'].values)

In [ ]:
#check if all is ok
impute_covid.isnull().sum()[impute_covid.isnull().sum()>0]

In [ ]:
#ok we have no problem see table data
impute_covid.info()

### The Worldwide confirmed, recovered, death and active confirmed 

In [ ]:
start_date = impute_covid.ObservationDate.min()
end_date = impute_covid.ObservationDate.max()
print('Novel Covid-19 information:\n 1. Start date = {}\n 2. End date = {}'.format(start_date, end_date))

In [ ]:
worldwide = impute_covid[impute_covid['ObservationDate'] == end_date]

In [ ]:
nb_country = len(worldwide['Country/Region'].value_counts()) # number country
worldwide['Country/Region'].value_counts()

In [ ]:
world = worldwide.groupby('Country/Region').sum()
world = world.sort_values(by=['Confirmed'], ascending=False)
world.style.background_gradient(cmap='viridis')

In [ ]:
print('================ Worldwide report ===============================')
print('== Information to {} on novel COVID-19 =========\n'.format(end_date))
print('Tota confirmed: {}\nTotal Deaths: {}\nTotal Recovered: {}\nTotal active confirmed: {}\n\
Total country Recorded: {} \n'.format(\
worldwide.Confirmed.sum(), worldwide.Deaths.sum(), worldwide.Recovered.sum(), worldwide.active_confirmed.sum(),\
                                     nb_country))
print('==================================================================')

In [ ]:
world.Confirmed.iplot(kind='bar', title='novel Covid-19 in the Worldwide', yTitle='Total cases', colors='blue',
                      logy=True)
#world.Confirmed.plot(kind='bar', title= 'novel Covid-19 in the Worldwide', figsize=(20,8), logy=True,legend=True)
#plt.ylabel('Total Cases')

In [ ]:
world.Recovered.iplot(kind='bar', title='novel Covid-19 in the Worldwide', yTitle='Total Recovered', \
colors='green', logy=True)
#world.Recovered.plot(kind='bar', title= 'novel Covid-19 in the Worldwide', figsize=(20,8), logy=True,\
 #                    colormap='Greens_r', legend=True)
#plt.ylabel('Total Recovered')

In [ ]:
world.Deaths.iplot(kind='bar', title='novel Covid-19 in the Worldwide', yTitle='Total Deaths', logy=True)
#world.Deaths.plot(kind='bar', title= 'novel Covid-19 in the Worldwide', figsize=(20,8), logy=True,\
 #                    colormap='Reds_r', legend=True)
#plt.ylabel('Total Deaths')

In [ ]:
world.active_confirmed.iplot(kind='bar', title='novel Covid-19 in the Worldwide', yTitle='Total  Active Cases',
colors='blue', logy=True)
#world.active_confirmed.plot(kind='bar', title= 'novel Covid-19 in the Worldwide', figsize=(20,8), logy=True,\
                        #    legend=True)
#plt.ylabel('Total  Active Cases')

## The 6 countries most affected by the novel covid-19 

In [ ]:
world_table = world.reset_index()

In [ ]:
x = world_table[world_table['Country/Region'] == 'France']
big_7 = world_table[world_table['Confirmed'] >= x.iloc[0,1]]

In [ ]:
big_7.style.background_gradient(cmap='viridis')

## Time

**Special China**

In [ ]:
china = impute_covid[impute_covid['Country/Region'] == 'Mainland China']

In [ ]:
chstar_date = china.ObservationDate.min()
chend_date = china.ObservationDate.max()

In [ ]:
print('Novel covid-19 China:\n start date = {}\n end date = {}'.format(chstar_date, chend_date))

In [ ]:
lastChina = china[china['ObservationDate'] == chend_date]
lastChina.head(2)

In [ ]:
print('================ China report ===================================')
print('== Information to {} on novel COVID-19 =========\n'.format(chend_date))
print('Tota confirmed: {}\nTotal Deaths: {}\nTotal Recovered: {}\nTotal active confirmed: {}\n'.format(\
lastChina.Confirmed.sum(), lastChina.Deaths.sum(), lastChina.Recovered.sum(), lastChina.active_confirmed.sum()))
print('==================================================================')

In [ ]:
province = lastChina.groupby('Province/State').sum()
province = province.sort_values(by=['Confirmed'], ascending=False)

In [ ]:
province.iplot(kind='bar', yTitle='Total patient',logy=True,
               title='China Province  with novel covid-19')
#province.plot(kind='bar', label='Confirmed',logy=True,figsize=(20,10), stacked=True,\
 #             title='China Province  with novel covid-19')
#plt.ylabel('Total patient')

In [ ]:
conf_china = china.groupby('ObservationDate')['Confirmed'].agg('sum')
rec_china = china.groupby('ObservationDate')['Recovered'].agg('sum')
dea_china = china.groupby('ObservationDate')['Deaths'].agg('sum')
ac_china = china.groupby('ObservationDate')['active_confirmed'].agg('sum')

In [ ]:
fig = plt.figure(figsize = (20,20))
fig.subplots_adjust(hspace=0.4, wspace=0.125)
dg = [conf_china,rec_china,dea_china,ac_china]
color = ['blue', 'green', 'red', 'blue']
for i in range(1, len(dg)+1):
    ax = fig.add_subplot(2,2,i)
    dg[i-1].iplot(kind='bar', color = color[i-1], legend=True)
    ax.set_ylabel('Total patient')

**ROW**

In [ ]:
rest_world = impute_covid[impute_covid['Country/Region'] != 'Mainland China']

In [ ]:
rest_world.head(2)

In [ ]:
print('Novel covid-19 ROW:\n start date = {}\n end date = {}'.format(rest_world.ObservationDate.min(),\
                    rest_world.ObservationDate.max()))

In [ ]:
row = rest_world[rest_world['ObservationDate'] == rest_world.ObservationDate.max()]

In [ ]:
print('================ ROW report =====================================')
print('== Information to {} on novel COVID-19 =========\n'.format(chend_date))
print('Tota confirmed: {}\nTotal Deaths: {}\nTotal Recovered: {}\nTotal active confirmed: {}\n'.format(\
row.Confirmed.sum(), row.Deaths.sum(), row.Recovered.sum(), row.active_confirmed.sum()))
print('==================================================================')

In [ ]:
obs_conf_world = rest_world.groupby('ObservationDate')['Confirmed'].aggregate([np.sum]) # confirmed obs
ac_conf_world = rest_world.groupby('ObservationDate')['active_confirmed'].aggregate([np.sum]) # last upd obs
patient_world_r = rest_world.groupby('ObservationDate')['Recovered'].aggregate([np.sum]) # lifetime 
patient_world_dea = rest_world.groupby('ObservationDate')['Deaths'].aggregate([np.sum]) # lifetime 

obs_conf_world.columns = ['Confirmed']
ac_conf_world.columns = ['active_onfirmed']
patient_world_r.columns = ['Recovered'] 
patient_world_dea.columns = ['Deaths'] 

In [ ]:
fig = plt.figure(figsize = (20,20))
fig.subplots_adjust(hspace=0.4, wspace=0.125)
dr = [obs_conf_world, patient_world_r, patient_world_dea, ac_conf_world]
color = ['blue', 'green', 'red', 'blue']
for i in range(1, len(dr)+1):
    ax = fig.add_subplot(2,2,i)
    dr[i-1].plot(ax=ax, kind='bar', color = color[i-1], legend=True)
    ax.set_ylabel('Total patient')

# Comparison China vs ROW

**N.B**

For me, a data scientist should not spend his time making beautiful curves even as it is the most important part in data science but to discover the knowledge that hides behind the data. these data take shape when they are transformed into curves (Visualization) and cannot speak. We make them speak when we interpret these curves. this is why the goal of this notebook, is to bring out the information hidden in the curves to be able to predict if the end of the covid 19 is closer in China or closer to the rest of the world.

In [ ]:
#time_obs.iplot(title='novel COVID-19 in the Worldwide', kind='bar')
time_obs = impute_covid.groupby('ObservationDate')['Confirmed'].aggregate([np.sum])
time_obs.columns = ['Confirmed']
g =time_obs.plot(figsize=(20,8), kind='bar', color='black')
conf_china.plot(ax=g, kind='bar', color = 'blue', legend=True)
plt.title('novel COVID-19 in the Worldwide: China(Blue) vs ROW(Black)')
plt.ylabel('Total patient')

the 2019-2020 coronavirus pandemic is an emerging infectious disease called COVID-19 caused by the coronavirus sars-cov-2, which begins in December 2019 in Wuhan, central China, and then spreads worldwide.

**Event**

1- **11 mars 2020**: WHO declare COVID-19 as pandemic. At that time, number of confirmed cases = 125000 (see figure)

2- **February 24, 2020** plus confirmed case of covid 19 reported in Italy.

3- **January 25**, the Chinese authority quarantines 59 million people i.e. the entire province of Hubei is quarantined. This is why we see the curve from 22 to 25 Jan the first confirmed cases.

4- **February 16, 2020** the authorities of the province of Hubei, reinforced this quarantine by limiting the circulation of transport and people to prevent the spread of the virus. you see the graph on that date; the curve turns towards saturation.
>Statement by the Chinese authorities in Hubei province: 
**"The measures to prevent and control the epidemic in our province have entered their decisive phase, but the situation remains extremely serious. And it is to effectively isolate the source of infection and to curb the risks of contagion that it has been decided to further tighten prophylaxis and control measures […] We count on the understanding and cooperation of the inhabitants of the province, as well as on their active participation in the fight against the epidemic."**

5- **February 2, 2020**, several countries declare the appearance of contamination cases which are: South Korea, Republic of Korea, Thailand, Japan, Taiwan, Hong Kong, Singapore, Vietnam, Australia, Canada, United States, Finland, Iran, Italy ... What bothers me here is that these countries especially Italy, South Korea, Iran, US are now among the 7 countries most affected in the world by covid-19. The question I ask myself is as follows: is it because the authorities of these countries did not take this disease seriously or the population refused to accept the change in lifestyle imposed by the state or inevitably the negligence on the part of the population makes announcements of the first cases declared by covid19 which means that Italy and the 6 other countries (see table above) are the most affected by this disease? (tell me if i'm wrong)

6- **January 24, 2020**, France repatriates its nationals from Wuhan.
January 30, 2020, first case declared in France and France becomes the first European country to be affected by covid19. At the time it was version I (SARS and MERS) of the coronavirus.
> After almost a month on February 29, when the world has more than 75,000 confirmed cases of covid19 where China has reached saturation (progressive control of the disease especially the province of Hubei see figure above) that France declares the ban on all confined gathering of more than 5000 people.

7-**Dec 31, 2019 to Feb 11, 2020**: Chinese population contaminated by SARS and MERS.

8- 1**2-13 February** sudden appearance of the new COVID19 and the number of confirmed cases increases and the number of deaths also increases (see figure.)


**source:**  https://www.notre-planete.info/actualites/4135-coronavirus-epidemie-Chine-Europe-monde

**source:**  https://en.wikipedia.org/wiki/2019%E2%80%9320_coronavirus_pandemic

In [ ]:
death_rate = impute_covid.groupby('ObservationDate')['Deaths'].aggregate([np.sum])
recovered_rate = impute_covid.groupby('ObservationDate')['Recovered'].aggregate([np.sum])
activecase_rate = impute_covid.groupby('ObservationDate')['active_confirmed'].aggregate([np.sum])
death_rate.columns = ['Death rate']
recovered_rate.columns = ['Recovered rate']
activecase_rate.columns = ['Active confirmed rate']

In [ ]:
h = recovered_rate.plot(figsize=(15.5, 5), colormap='Greens_r', kind='bar', legend=True) 
rec_china.plot(ax=h, color='yellow', legend=True, kind='bar')
plt.title('novel COVID-19 in the Worldwide: China(yellow) vs ROW(green)')
plt.ylabel('Total patient')

more chinese people recovered the health after affected by covid 19 and continue to grow that means all policy the chinese authorities implement to fight again this disease are better and  better efficient

In [ ]:
df = death_rate.plot(figsize=(15.5, 5), colormap='Reds_r', kind='bar', legend=True)
dea_china.plot(ax=df, color='black', kind='bar', legend=True)
plt.title('novel COVID-19 in the Worldwide: China(Black) vs ROW(Red) ')
plt.ylabel('Total patient')

also again better and better

In [ ]:
q = activecase_rate.plot(figsize=(15.5, 5),  color='black', kind='bar', legend=True)
ac_china.plot(kind='bar', color='blue', legend=True, ax=q)
plt.title('novel COVID-19 in the Worldwide: China(Blue) vs ROW(Black)')
plt.ylabel('Total patient')

When you see this graph, there exist more relevant information. two relevant information are:

1- **ALL policy that chinese authorities are implemented allow to control covid19**

2- **The end of covid-19 in china is coming if another disease does not come**

**short conclusion**

the growth or decline of covid 19 disease will depend on the effectiveness of the policy put in place to combat this disease. So the curve does show us the correlation that exists.

In [ ]:
from scipy import stats, linalg

def partial_corr(C):
    """
    Returns the sample linear partial correlation coefficients between pairs of variables in C, controlling 
    for the remaining variables in C.
    Parameters
    ----------
    C : array-like, shape (n, p)
        Array with the different variables. Each column of C is taken as a variable
    Returns
    -------
    P : array-like, shape (p, p)
        P[i, j] contains the partial correlation of C[:, i] and C[:, j] controlling
        for the remaining variables in C.
    """
    
    C = np.asarray(C)
    p = C.shape[1]
    P_corr = np.zeros((p, p), dtype=np.float)
    for i in range(p):
        P_corr[i, i] = 1
        for j in range(i+1, p):
            idx = np.ones(p, dtype=np.bool)
            idx[i] = False
            idx[j] = False
            beta_i = linalg.lstsq(C[:, idx], C[:, j])[0]
            beta_j = linalg.lstsq(C[:, idx], C[:, i])[0]

            res_j = C[:, j] - C[:, idx].dot( beta_i)
            res_i = C[:, i] - C[:, idx].dot(beta_j)

            corr = stats.pearsonr(res_i, res_j)[0]
            P_corr[i, j] = corr
            P_corr[j, i] = corr

    return P_corr

In [ ]:
def growth_rate(data=None):
    x = []
    x.append(0)
    for i in range(data.shape[0]-1):
        a = data.iloc[i+1]-data.iloc[i]
        x.append(a/data.iloc[i])
        
    return np.array(x)

In [ ]:
def ratio_recovered_death(reco = None, dea = None ):
    """       
        reco is the recovered rate data
        dea is the deaths rate data
    """
    
    x = []
    x.append(0)
    
    if (reco.shape[0] != dea.shape[0]):
        print('ratio is not working. See shape of your data recovered or death')
        
        return None
        
    else:
        
        for j in range(reco.shape[0] - 1):
            
            a = reco.iloc[j+1] - reco.iloc[j]
            b = dea.iloc[j+1] - dea.iloc[j]
            
            c = a/(b+1)
            x.append(c)  
            
    return  np.array(x)

## Correlation between growth rate and types of mitigation across countries (china vs ROW/ Italy, France, Iran, South Korea, US, ...)

### China 

>Statement by the Chinese authorities in Hubei province: 
**"The measures to prevent and control the epidemic in our province have entered their decisive phase, but the situation remains extremely serious. And it is to effectively isolate the source of infection and to curb the risks of contagion that it has been decided to further tighten prophylaxis and control measures […] We count on the understanding and cooperation of the inhabitants of the province, as well as on their active participation in the fight against the epidemic."**

In [ ]:
# we define a correlation and partial correlation
china.corr().iplot(kind='heatmap', colorscale='Reds', title="Feature correlation Matrix")

In [ ]:
nch = list(set(china.columns) - set(['SNo', 'ObservationDate', 'Province/State', 'Country/Region',
       'Last Update']))
pcoray_china = china[nch].values 
corrpartial_china = pd.DataFrame(partial_corr(pcoray_china), columns=nch, index=nch)
corrpartial_china.head()

this result tell us that if we take for example deaths and recovered and you fix all other feature, recovered and deaths are more associate between them. And recovered is opposite direction on deaths. we can see a graph.

In [ ]:
# plot to see
sns.pairplot(china[nch])

When we see recovered with other feature we realize that recovered and death, confirmed have exponential shape where confirmed become satured and also death. 

For active confirmed between [0, 1000], the curve increase after it fall. that means the people who are current confirmed become recovered

In [ ]:
#now we start to compute the ac growth rate and ratio recovered-death in china
data_china = china.groupby('ObservationDate')[['active_confirmed', 'Recovered', 'Deaths']].agg('sum')

In [ ]:
ac_growth_rate_china = growth_rate(data_china['active_confirmed'])

In [ ]:
rate_china = pd.DataFrame(ac_growth_rate_china, columns=['AC_growth_rate'], index=data_china.index)
rate_china.iplot(title='Active Confirmed Growth Rate in China', yTitle='Growth rate', legend=True)
#plt.ylabel('Growth rate')
#plt.title('Active Confirmed Growth Rate')

In [ ]:
ratio_reco_dea_china = ratio_recovered_death(data_china.Recovered, data_china.Deaths)

In [ ]:
ratio_china = pd.DataFrame(ratio_reco_dea_china, columns=['Ratio_recovered_death'], index = data_china.index)
ratio_china.iplot(title='Ratio Recovered Deaths in China',yTitle='ratio recovered death', legend=True)
#plt.ylabel('ratio recovered death')
#plt.title('Ratio Recovered Deaths in China')

In [ ]:
rest_world.columns

## US
> **Trump issues ‘Coronavirus Guidelines’ for next 15 days to slow pandemic**

>San Francisco Mayor **London Breed** said, “**Effective at midnight, San Francisco will require people to stay home except for essential needs. Necessary government functions & essential stores will remain open.**” 

source https://www.cnbc.com/2020/03/16/coronavirus-live-updates-us-retail-closures-could-explode-irans-deaths-surge.html

In [ ]:
us = rest_world[rest_world['Country/Region'] == 'US']

In [ ]:
sns.pairplot(us[nch])

In [ ]:
us_data = us.groupby('ObservationDate')[nch].agg('sum')

In [ ]:
us_growth_rate = pd.DataFrame(growth_rate(us_data.active_confirmed), columns=['growth_rate'], index=us_data.index)
us_ratio = pd.DataFrame(ratio_recovered_death(us_data.Recovered, us_data.Deaths),
                        columns=['ratio_recovered_death'],
                       index=us_data.index)

In [ ]:
us_growth_rate.iplot(title='Active Confirmed Growth Rate in US', yTitle='Growth rate', legend=True)

In [ ]:
us_ratio.iplot(title='Ratio Recovered Deaths in US', yTitle='ratio recovered death', legend=True)

## France

>French President **Emmanuel Macron** issued nearly identical directives, saying French residents **’movements would be severely limited for 15 days**.Macron also said he would be closing the country’s borders starting Tuesday afternoon.

source : https://www.cnbc.com/2020/03/16/coronavirus-live-updates-us-retail-closures-could-explode-irans-deaths-surge.html

In [ ]:
france = rest_world[rest_world['Country/Region'] == 'France' ]

In [ ]:
sns.pairplot(france[nch])

In [ ]:
french_data = france.groupby('ObservationDate')[nch].agg('sum')

In [ ]:
french_growth_rate = pd.DataFrame(growth_rate(french_data.active_confirmed), columns=['growth_rate'] ,
                                  index=french_data.index )
ratio_french =pd.DataFrame(ratio_recovered_death(french_data.Recovered, french_data.Deaths),
                            columns=['ratio_recovered_death'], index=french_data.index)

In [ ]:
french_growth_rate.iplot(title='Active Confirmed Growth Rate in France', yTitle='Growth rate', legend=True)

In [ ]:
ratio_french.iplot(title='Ratio Recovered Deaths in France', yTitle='ratio recovered death', legend=True)

**Conclusion: As you see yourself, there exist the Correlation between growth rate and types of mitigation. The eradication of covid 19 in a country depends on its policy for combating this disease and the way in which this policy is implemented in immediate time and in space.**

### where and when will covid 19 be eradicated?

Before to find the end of disease. We need to determinate some notion:

- seasonality of confirmed case
- weekly recovered rate
- weekly death rate
- weekly life increase or decrease

N.B: We know that time series can be expressed as ts = trend + seasonal + cyclical + irregular. But here we study only a seasonal part that contain a confirmed cases, recovered, death and active c feature.

## Seasonality of covid19 in USA**

In [ ]:
us['Province/State'].nunique()

In [ ]:
us_state = us.groupby(['Province/State','ObservationDate'])[nch].agg('sum')
us_observation = us.groupby('ObservationDate')[nch].agg('sum')

In [ ]:
us_observation.head()

In [ ]:
weekly_us = us_observation.resample('W').agg('mean')
weekly_us.head()

In [ ]:
from statsmodels.tsa import seasonal

In [ ]:
decompose_us = seasonal.seasonal_decompose(us_observation.Confirmed, model='additive', period=7)

In [ ]:
decompose_us.seasonal.iplot(xTitle='Date', yTitle='Confirmed case', title="Seasonal of Covid 19 in USA")

**period = 7 days**  

In [ ]:
decompose_us.trend.iplot(xTitle='Date', yTitle='Confirmed case', title="Trend of Covid 19 in USA")

# Weekly recovered rate, death rate, life increase or decrease in USA

**weekly recovered rate** is the ratio of the average weekly number of people cured over the average weekly number of people infected with covid 19.

**weekly death rate** is the ratio of the average weekly number of dead people over the average weekly number of people infected with covid 19.

**life increase or decrease** is the difference between weekly recovered rate and weekly death rate.(increase when is positive and decrease when is negative.) 

In [ ]:
weekly_recovered_rate = weekly_us.Recovered/weekly_us.Confirmed
weekly_death_rate = weekly_us.Deaths/weekly_us.Confirmed
life_inc_dec = weekly_recovered_rate - weekly_death_rate

In [ ]:
weekly_recovered_rate.iplot(xTitle='Date', yTitle='Recovered rate', title='Weekly Recovered rate in USA',
                            color='green')

In [ ]:
weekly_death_rate.iplot(xTitle='Date', yTitle='Death rate', title='Weekly Death rate in USA',
                            color='black')

In [ ]:
life_inc_dec.iplot(xTitle='Date', yTitle='Life', title='Life increase or decrease in USA',
                            color='blue')

# Can we find the model that explain well the covid 19 disease control in China?

**We will look for this Model** 

We know that covid19 have four variable **ConfirmedCases, CurrentConfirmedCases, Recovered and Deaths**. how can we obtain the dynamics system equation for this variable? 

To answer this question, we are going to use the SIR Model:

The SIR model is a classic model in epidemiology, it contain three subpopulations, the susceptibles **S**, the infectives **I** and removed individuals **R**. In this context of covid19:

> Susceptiles is people containment

> Infective is currentConfirmedCases

> Recovered is recovered and deaths

The susceptible can become infective, and the infectives can become removed, but no other transitions are considered. Diagrammatically.

**$ S \rightarrow I \rightarrow R$**

The population $N = S + I + R$ remains constant. The model describes the movement between the classes by the system of differential equations.

> $\dfrac{dS}{d\tau} = -\beta IS$, $\dfrac{dI}{d\tau} = \beta IS -\gamma I$, $\dfrac{dR}{d\tau} = \gamma I$

In terms of fraction $x = \dfrac{S}{N}, y = \dfrac{I}{N}, z = \dfrac{R}{N}$, and the rescaled time variable $t = \gamma\tau$, the equations become

> $\dfrac{dx}{dt} = -R_{0}xy$, $\dfrac{dy}{dt} = R_{0}xy - y$, $\dfrac{dz}{dt} = y$, where $R_{0} = \dfrac{\beta N}{\gamma}$ it is called **reproduction ratio**

We can use only two equations to define  a system 

>  $\dfrac{dx}{dt} = -R_{0}xy$, $\dfrac{dy}{dt} = R_{0}xy - y$, and find $ z = 1 - x - y$. Mark region $D = \{(x,y)\in \mathbb{R}^2| 0 \leq x,y \leq 1, x + y \leq 1\}$ in the (x,y)-plane.

Okay, this is a simple literature for SIR Model. By this model, we define a model that it can explain the behavior of confirmed cases, current confirmed cases and (recovered + death) in China.

## SIR Modified for confirmed cases, current confirmed cases and (recovered and deaths)

> x is a susceptible fraction

> y is a infectives fraction

> z is a recovered fraction

We know that $z = 1 - x - y$. So,  the confirmed case fraction is **$ x^{c} = 1 - x$**. 

We have **$z = x^{c} - y$**, that is similar to **recovered+deaths = confirmed case - current confirmed case**

We can now modified a SIR Model that become a new model called SIR modified Model for COVID19 in China.

With $ x = 1 - x^{c}$, we have: 

>$\dfrac{dx^{c}}{dt} = R_{0}(1 - x^{c})y$,$\qquad$ $\dfrac{dy}{dt} = R_{0}y(1-x^{c}) - y$,$\qquad$  $z = x^{c} - y$

We are going to do some simulation of this model to see if it match well.

In [ ]:
''' 
	X,Y = integrate(F,x,y,xStop,h).
	4th-order Runge-Kutta method for solving the initial value problem 
	{y} = {y[0],y[1],...,y[n-1]}
	x,y = initial conditions
	xStop = terminal value of x
	h = increment of x used in integration
	F = user-supplied function that returns the     
    array F(x,y) = {y'[0], y'[1],...,y'[n-1]} 
'''

def integrate(F,x,y,xStop,h):
    
    def run_kut4(F, x, y, h):
        
        K0 = h*F(x,y)
        K1 = h*F(x + h/2.0, y + K0/2.0)
        K2 = h*F(x + h/2.0, y + K1/2.0)
        K3 = h*F(x + h, y + K2)
        
        return (K0 + 2.0*K1 + 2.0*K2 + K3)/6.0
        
    X = [ ]
    Y = [ ]
    
    X.append(x)
    Y.append(y)
    
    while x < xStop:
        
        h = min(h,xStop - x)
        y = y + run_kut4(F,x,y,h)
        x = x + h
        X.append(x)
        Y.append(y)
    
    return np.array(X), np.array(Y)

In [ ]:
"""
SIR Modified Model
x = S/N; y = I/N; z = R/N
x + y + z = 1 => 1 - xc + y + z = 1 => xc = y + z

dxc/dt = R0*(1- xc)*y
dy/dt = y*R0*(1-xc) - y

z = xc - y

"""

global R0

def F(t, y):

    u = y.copy()

    u[0] = R0*(1-y[0])*y[1]
    u[1] = y[1]*(R0*(1-y[0]) - 1)

    return u

In [ ]:
t = 0.0
tmax = 4  #
dt = 0.01  # 
a = np.array([0.01, 0.008])

In [ ]:
R0 = 13.5
T, Y = integrate(F,t, a, tmax, dt)

In [ ]:
sirModified = pd.DataFrame()
sirModified['Time'] = T
sirModified['x_confirmedCases'] = Y[:, 0]
sirModified['y_currentConfirmedCases'] = Y[:, 1]
sirModified['z_Recovered'] = Y[:, 0] - Y[:, 1]

In [ ]:
sirModified.head()

In [ ]:
sirModified.plot(x='Time', figsize=(15,5))
plt.ylabel('fraction of three class')
plt.title('SIR Modified Model simulation for covid19 in China')

In [ ]:
ax = conf_china.plot(figsize=(15,5), legend = True)
ac_china.plot(ax=ax, legend=True)
rec_china.plot(ax=ax, legend = True)
plt.title('real curve of covid19 in China')

In [ ]:
fig = plt.figure(figsize=(15,5))
fig.subplots_adjust(hspace=0.4, wspace=0.2)
ax1 = fig.add_subplot(1,2,1)
ax2 = fig.add_subplot(1,2,2)
sirModified.plot(x='x_confirmedCases', y='y_currentConfirmedCases', ax = ax1)
ax1.set_ylabel('y_currentConfirmedCases')
ax1.set_title('phase plane simulation with R0=4.5')

china.plot(kind='scatter', x='Confirmed', y='active_confirmed', ax=ax2)
ax2.set_title('phase plane real')

In [ ]:
figo = plt.figure(figsize=(15,5))
figo.subplots_adjust(hspace=0.4, wspace=0.4)
axo1 = figo.add_subplot(1,2,1)
axo2 = figo.add_subplot(1,2,2)
sirModified.plot(x='x_confirmedCases', y='z_Recovered', ax = axo1)
axo1.set_ylabel('z_Recovered')
axo1.set_title('phase plane simulation with R0={}'.format(R0))

china.plot(kind='scatter', x='Confirmed', y='Recovered', ax=axo2)
axo2.set_title('phase plane real')

In [ ]:
figu = plt.figure(figsize=(15,5))
figu.subplots_adjust(hspace=0.4, wspace=0.4)
axu1 = figu.add_subplot(1,2,1)
axu2 = figu.add_subplot(1,2,2)
sirModified.plot(y='y_currentConfirmedCases', x='z_Recovered', ax = axu1)
axu1.set_ylabel('y_currentConfirmedCases')
axu1.set_title('phase plane simulation with R0= {}'.format(R0))

china.plot(kind='scatter', y='active_confirmed', x='Recovered', ax=axu2)
axu2.set_title('phase plane real')

**We see that SIR modified Model can as well explain the covid19 control in China**

### Upnext